# The end result of this exercise should be a file named acquire.py.

## Getting to Know API

In [1]:
import requests

base_url = 'https://python.zach.lol/documentation'

In [2]:
response = requests.get(base_url)
print(response.json()['payload'])


The API accepts GET requests for all endpoints, where endpoints are prefixed
with

    /api/{version}

Where version is "v1"

Valid endpoints:

- /stores[/{store_id}]
- /items[/{item_id}]
- /sales[/{sale_id}]

All endpoints accept a `page` parameter that can be used to navigate through
the results.



In [3]:
response = requests.get('https://python.zach.lol/api/v1/items')

data = response.json()
data.keys()

dict_keys(['payload', 'status'])

In [4]:
data['payload'].keys()

dict_keys(['items', 'max_page', 'next_page', 'page', 'previous_page'])

In [5]:
data['payload']['items'][:2]

[{'item_brand': 'Riceland',
  'item_id': 1,
  'item_name': 'Riceland American Jazmine Rice',
  'item_price': 0.84,
  'item_upc12': '35200264013',
  'item_upc14': '35200264013'},
 {'item_brand': 'Caress',
  'item_id': 2,
  'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct',
  'item_price': 6.44,
  'item_upc12': '11111065925',
  'item_upc14': '11111065925'}]

### 1. Using the code from the lesson as a guide, create a dataframe named items that has all of the data for items.

In [6]:
import pandas as pd

In [7]:
df_items = pd.DataFrame(data['payload']['items'])
df_items.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


### 2. Do the same thing, but for stores.

In [8]:
response = requests.get('https://python.zach.lol/api/v1/stores')

data = response.json()
data.keys()

dict_keys(['payload', 'status'])

In [9]:
df_stores = pd.DataFrame(data['payload']['stores'])
df_stores.head()

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


### 3. Extract the data for sales. There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

In [10]:
response = requests.get('https://python.zach.lol/api/v1/sales')

data = response.json()
data.keys()

dict_keys(['payload', 'status'])

In [11]:
print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

max_page: 183
next_page: /api/v1/sales?page=2


In [13]:
base_url = 'https://python.zach.lol'

api_url = base_url + '/api/v1/'
response = requests.get(api_url + 'sales')
data = response.json()
    
# create list from 1st page
output = data['payload']['sales']

# loop through the pages and add to list
while data['payload']['next_page'] != None:
    
    response = requests.get(base_url + data['payload']['next_page'])
    data = response.json()
    output.extend(data['payload']['sales'])
    
df_sales = pd.DataFrame(output)

In [14]:
df_sales

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1
...,...,...,...,...,...
912995,50,63.0,"Wed, 27 Dec 2017 00:00:00 GMT",912996,10
912996,50,59.0,"Thu, 28 Dec 2017 00:00:00 GMT",912997,10
912997,50,74.0,"Fri, 29 Dec 2017 00:00:00 GMT",912998,10
912998,50,62.0,"Sat, 30 Dec 2017 00:00:00 GMT",912999,10


### 4. Save the data in your files to local csv files so that it will be faster to access in the future.

In [15]:
df_items.to_csv("items_df.csv")
df_stores.to_csv("stores_df.csv")
df_sales.to_csv("sales_df.csv")

### 5. Combine the data from your three separate dataframes into one large dataframe.

In [16]:
df_items.head(1)

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013


In [17]:
df_stores.head(1)

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253


In [18]:
df_sales.head(1)

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1


In [19]:
# left join sales and stores
df = pd.merge(df_sales, df_stores, left_on='store', right_on='store_id').drop(columns={'store'})
df.head(2)

,item,sale_amount,sale_date,sale_id,store_address,store_city,store_id,store_state,store_zipcode
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253


In [21]:
# left join the joined df to the items
df = pd.merge(df, df_items, left_on='item', right_on='item_id').drop(columns={'item'})
df.head(2)

,sale_amount,sale_date,sale_id,store_address,store_city,store_id,store_state,store_zipcode,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013


### 6. Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

In [22]:
url = 'https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv'

open_power_df = pd.read_csv(url)

In [23]:
open_power_df.head()

,Date,Consumption,Wind,Solar,Wind+Solar
0,2006-01-01,1069.184,NaN,NaN,NaN
1,2006-01-02,1380.521,NaN,NaN,NaN
2,2006-01-03,1442.533,NaN,NaN,NaN
3,2006-01-04,1457.217,NaN,NaN,NaN
4,2006-01-05,1477.131,NaN,NaN,NaN


### 7. Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.

In [ ]:
# Acquire.py Functions Using REST API that Returns JSON

import requests
import os
import numpy as np
import pandas as pd

######################################### Items DF ########################################

def get_items():
    
    if os.path.isfile('items_df.csv') == False:
        
        base_url = 'https://python.zach.lol'
        api_url = base_url + '/api/v1/items'
        response = requests.get(api_url)
        data = response.json()
        df_items = pd.DataFrame(data['payload']['items'])
        
    else:
        # Reads the csv saved from above, and assigns to the df variable
        df_items = pd.read_csv('items_df.csv', index_col=0)
        
    return df_items

######################################### Stores DF #######################################

def get_stores():
    
    if os.path.isfile('stores_df.csv') == False:
        
        base_url = 'https://python.zach.lol'
        api_url = base_url + '/api/v1/stores'
        response = requests.get(api_url)
        data = response.json()
        df_stores = pd.DataFrame(data['payload']['stores'])
        
    else:
        # Reads the csv saved from above, and assigns to the df variable
        df_stores = pd.read_csv('stores_df.csv', index_col=0)

    return df_stores
    
######################################### Sales DF ########################################

def get_sales():
    
    if os.path.isfile('sales_df.csv') == False:
        
        base_url = 'https://python.zach.lol'

        api_url = base_url + '/api/v1/'
        response = requests.get(api_url + 'sales')
        data = response.json()
    
        # create list from 1st page
        output = data['payload']['sales']

        # loop through the pages and add to list
        while data['payload']['next_page'] != None:
    
            response = requests.get(base_url + data['payload']['next_page'])
            data = response.json()
            output.extend(data['payload']['sales'])
    
        df_sales = pd.DataFrame(output)
        
    else:
        # Reads the csv saved from above, and assigns to the df variable
        df_sales = pd.read_csv('sales_df.csv', index_col=0)
        
    return df_sales

####################################### Joined Sales DF ########################################

def get_joined_sales():
    
    if os.path.isfile('joined_sales_df.csv') == False:
        
        df_items = get_items()
        df_stores = get_stores()
        df_sales = get_sales()
    
        # left join sales and stores
        df = pd.merge(df_sales, df_stores, left_on='store', right_on='store_id').drop(columns={'store'})
    
        # left join the joined df to the items
        df = pd.merge(df, df_items, left_on='item', right_on='item_id').drop(columns={'item'})
        
    else:
        # Reads the csv saved from above, and assigns to the df variable
        joined_sales_df = pd.read_csv('joined_sales_df.csv', index_col=0) 
        
    return joined_sales_df

######################################### Open Power DF ########################################

def get_open_power():

    if os.path.isfile('open_power_df.csv') == False:
        
        url = 'https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv'
        open_power_df = pd.read_csv(url)
        
    else:
        # Reads the csv saved from above, and assigns to the df variable
        open_power_df = pd.read_csv('open_power_df.csv', index_col=0)
    
    return open_power_df